In [86]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim

In [87]:
import glob
path = "./../dist/char*.txt"
contents=[]
for filename in glob.glob(path):
    with open(filename, 'r') as f:
        for line in f:
            contents.append(line.strip('\n'))
print(len(contents))
i=0
while (i<len(contents)-1):
    contents[i]=float(contents[i])
    i+=1
    if ((i+1)%15==0):
        i=i+1

1635


In [88]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [89]:
from random import randint as r

In [90]:
i=0
for a in range(0, 49):
    i=r(0, (len(contents)/15-1))
    tt=torch.tensor(contents[(a*15):(a*15+14)])
    print(a, ' : ', tt)

0  :  tensor([3.7400e+02, 1.9200e+02, 1.6980e+03, 8.6543e+02, 1.7095e+00, 3.4959e+00,
        2.7036e+01, 2.3128e+01, 1.3257e+05, 1.8040e+04, 1.0058e+01, 1.3623e+00,
        3.5233e-02, 6.8635e-02])
1  :  tensor([1.6800e+02, 1.8000e+02, 6.1423e+02, 6.6076e+02, 1.7579e+00, 1.7579e+00,
        2.1776e+01, 2.6145e+01, 3.0122e+05, 2.6802e+04, 1.8465e+01, 1.6489e+00,
        7.8435e-02, 7.2972e-02])
2  :  tensor([1.6800e+02, 2.1600e+02, 3.5532e+04, 2.3201e+04, 3.4959e+00, 3.3456e+00,
        2.4845e+01, 2.1307e+01, 3.2577e+03, 4.9163e+02, 1.1552e+01, 8.8423e-01,
        7.8440e-02, 6.1009e-02])
3  :  tensor([1.7000e+02, 1.9600e+02, 1.0244e+03, 1.1821e+03, 4.2629e+00, 4.6510e+00,
        1.8965e+01, 1.6539e+01, 9.7704e+04, 7.7159e+03, 9.8711e+00, 7.7954e-01,
        5.7453e-02, 4.9622e-02])
4  :  tensor([1.3700e+02, 2.0300e+02, 8.2433e+02, 1.2244e+03, 4.7254e+00, 4.8075e+00,
        1.6646e+01, 1.8780e+01, 9.7918e+04, 5.9891e+03, 9.8917e+00, 6.0503e-01,
        6.9876e-02, 4.8330e-02])
5  : 

In [91]:
import time

In [92]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(in_features=14, out_features=14, bias=True)
        self.fc2 = nn.Linear(in_features=14, out_features=4, bias=True)
        #self.out = nn.Linear(in_features=14, out_features = 4, bias=True)
        
    def forward(self, t):
        #1st layer
        t = self.fc1(t)
        #2nd layer
        t = self.fc2(t)
        #output layer
        #t = self.out(t)
        
        return t

In [93]:
net=Network()
params=list(net.parameters())
criterion = nn.MSELoss()
net.zero_grad()
optimizer = optim.SGD(net.parameters(), lr=0.0000000005)
lossDuplicationCounter = 0
prevloss=0
loss = 4
i=0

In [94]:
start=time.time()
while ((loss > 0.001) and (lossDuplicationCounter<10)):
    a=r(0, (len(contents)/15-1))
    chars = torch.tensor(contents[(a*15):(a*15+14)])
    optimizer.zero_grad()   # zero the gradient buffers
    out = net(chars)
    if (contents[a*14+15]=='Marcin'):
        target = torch.tensor([1., 0., 0., 0.])
    elif (contents[a*14+15]=='Bartek'):
        target = torch.tensor([0., 1., 0., 0.])
    elif (contents[a*14+15]=='Maciek'):
        target = torch.tensor([0., 0., 1., 0.,])
    else:
        target = torch.tensor([0., 0., 0., 1.,])
    loss = criterion(out, target)
    if prevloss == loss:
        lossDuplicationCounter +=1
        print('loss duplicate #', lossDuplicationCounter)
    else:
        lossDuplicationCounter = 0
    prevloss = loss
    loss.backward(retain_graph=True)
    #loss.backward()
    optimizer.step()    # Does the update
    if i%1000==0:
        print("\n\niteration: ", i, "\nloss: ", loss, '\n\tOutput:\n', out)
    i+=1
print("Learning time: \n\t", time.time()-start)
torch.cuda.empty_cache()



iteration:  0 
loss:  tensor(66751988., grad_fn=<MseLossBackward>) 
	Output:
 tensor([-3089.6123,  5282.0879,  1153.5486, 15108.3203],
       grad_fn=<AddBackward0>)


iteration:  1000 
loss:  tensor(1201.7990, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ 11.5529,  -1.5877, -31.5833, -59.6111], grad_fn=<AddBackward0>)


iteration:  2000 
loss:  tensor(2071.6899, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ 60.7431, -42.8767, -42.3128, -30.1167], grad_fn=<AddBackward0>)


iteration:  3000 
loss:  tensor(620.5328, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-33.3064,  30.4814,   9.9914, -17.5437], grad_fn=<AddBackward0>)


iteration:  4000 
loss:  tensor(909.8060, grad_fn=<MseLossBackward>) 
	Output:
 tensor([  3.6790, -15.6043,   9.0378,  58.4501], grad_fn=<AddBackward0>)


iteration:  5000 
loss:  tensor(2756.3538, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ 11.5702, -20.4334,  16.5932, 101.9885], grad_fn=<AddBackward0>)


iteration:  6000 
loss:  tensor(301.1143, grad_fn=<M



iteration:  54000 
loss:  tensor(15.5345, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ 0.1688, -7.5334,  0.0632, -1.3136], grad_fn=<AddBackward0>)


iteration:  55000 
loss:  tensor(0.9156, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ 0.4138,  1.4541, -0.1927, -0.1574], grad_fn=<AddBackward0>)


iteration:  56000 
loss:  tensor(1.6397, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ 0.6006,  1.3525,  0.0463, -1.0897], grad_fn=<AddBackward0>)


iteration:  57000 
loss:  tensor(2.8776, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-0.8665,  2.1111,  0.8245,  3.3713], grad_fn=<AddBackward0>)


iteration:  58000 
loss:  tensor(0.8001, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-0.6006,  1.4293, -0.1363,  0.1179], grad_fn=<AddBackward0>)


iteration:  59000 
loss:  tensor(8.6288, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-0.6073,  5.0640,  1.0633,  3.7152], grad_fn=<AddBackward0>)


iteration:  60000 
loss:  tensor(0.1988, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ 0.4761,  

In [95]:
i=0
for filename in glob.glob(path):
    #a=r(0, (len(contents)/15-1))
    chars = torch.tensor(contents[(i*15):(i*15+14)])
    print(str(net(chars)).strip(", grad_fn=<AddBackward0>)").strip("tensor("), "\t : ",contents[i*15+14])
    i=i+1

[ -0.8419, -29.8164,  -0.9564,   4.0439] 	 :  Maciek
[ 0.8882, 34.7605,  5.8932,  4.3201] 	 :  Maciek
[ 0.3619,  2.4204, -0.5053, -0.4203] 	 :  Maciek
[-0.2969, -5.6377,  0.5202,  0.6555] 	 :  Marcin
[ 0.1018, -0.3584,  0.8493,  0.4865] 	 :  Marcin
[0.0484, 1.4585, 1.2443, 0.7185] 	 :  Marcin
[-0.3481, -5.1987,  0.8425,  1.0701] 	 :  Marcin
[-0.5607, -8.3938, -0.2576,  1.8536] 	 :  Bartek
[-0.1673,  6.1702,  1.0975,  2.5996] 	 :  Bartek
[-0.3759, -2.5265,  0.0269,  1.9440] 	 :  Bartek
[-0.2936, -1.4823, -0.2322,  1.8550] 	 :  Bartek
[-0.3619, -2.8664, -0.1761,  1.8569] 	 :  Bartek
[-0.2879, -3.9325, -0.3127,  1.9177] 	 :  Bartek
[-0.4021, -7.1102,  0.0248,  2.1638] 	 :  Bartek
[-0.2234,  0.3945,  0.0435,  2.1155] 	 :  Bartek
[-0.1805,  2.5387,  0.4962,  2.3592] 	 :  Bartek
[-0.1631,  3.6906,  0.2181,  2.1375] 	 :  Bartek
[-0.1565,  2.9754,  0.1786,  2.1769] 	 :  Bartek
[-0.1308,  4.6806,  0.7763,  2.5353] 	 :  Bartek
[ -0.5475, -11.3230,   0.6765,   0.3956] 	 :  Marcin
[-0.1479, -7.113

In [96]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
fc1.weight 	 torch.Size([14, 14])
fc1.bias 	 torch.Size([14])
fc2.weight 	 torch.Size([4, 14])
fc2.bias 	 torch.Size([4])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 5e-10, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [2570222566064, 2570222567144, 2570222566712, 2570222565848]}]


In [97]:
torch.save(net, "net.dat")
torch.save(net.state_dict(), "netstate.dat")

In [98]:
haba = torch.load("net.dat")
baba = Network()
baba.load_state_dict(torch.load("netstate.dat"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])